Web a la pagina con informacion del dataset: 
https://www.kaggle.com/datasets/alexteboul/diabetes-health-indicators-dataset?

In [14]:
import pandas as pd

# Ruta del archivo CSV
archivo_csv = "../data/diabetes_012_health_indicators_BRFSS2015.csv"

# Cargar el CSV en un DataFrame
df = pd.read_csv(archivo_csv)

# Mostrar las primeras filas
df.head(10)

,Diabetes_012,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,...,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
0,0.0,1.0,1.0,1.0,40.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,5.0,18.0,15.0,1.0,0.0,9.0,4.0,3.0
1,0.0,0.0,0.0,0.0,25.0,1.0,0.0,0.0,1.0,0.0,...,0.0,1.0,3.0,0.0,0.0,0.0,0.0,7.0,6.0,1.0
2,0.0,1.0,1.0,1.0,28.0,0.0,0.0,0.0,0.0,1.0,...,1.0,1.0,5.0,30.0,30.0,1.0,0.0,9.0,4.0,8.0
3,0.0,1.0,0.0,1.0,27.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,2.0,0.0,0.0,0.0,0.0,11.0,3.0,6.0
4,0.0,1.0,1.0,1.0,24.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,2.0,3.0,0.0,0.0,0.0,11.0,5.0,4.0
5,0.0,1.0,1.0,1.0,25.0,1.0,0.0,0.0,1.0,1.0,...,1.0,0.0,2.0,0.0,2.0,0.0,1.0,10.0,6.0,8.0
6,0.0,1.0,0.0,1.0,30.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,3.0,0.0,14.0,0.0,0.0,9.0,6.0,7.0
7,0.0,1.0,1.0,1.0,25.0,1.0,0.0,0.0,1.0,0.0,...,1.0,0.0,3.0,0.0,0.0,1.0,0.0,11.0,4.0,4.0
8,2.0,1.0,1.0,1.0,30.0,1.0,0.0,1.0,0.0,1.0,...,1.0,0.0,5.0,30.0,30.0,1.0,0.0,9.0,5.0,1.0
9,0.0,0.0,0.0,1.0,24.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,2.0,0.0,0.0,0.0,1.0,8.0,4.0,3.0


Hacemos un primer estudio de los tipos de datos, ya que en su mayoria son valores numéricos, pero pueden existir variables categoricas, y tipos que debamos corregir.

In [15]:
print("Tipos de datos actuales:\n")
print(df.dtypes)

Tipos de datos actuales:

Diabetes_012            float64
HighBP                  float64
HighChol                float64
CholCheck               float64
BMI                     float64
Smoker                  float64
Stroke                  float64
HeartDiseaseorAttack    float64
PhysActivity            float64
Fruits                  float64
Veggies                 float64
HvyAlcoholConsump       float64
AnyHealthcare           float64
NoDocbcCost             float64
GenHlth                 float64
MentHlth                float64
PhysHlth                float64
DiffWalk                float64
Sex                     float64
Age                     float64
Education               float64
Income                  float64
dtype: object


In [16]:
# Mostrar valores únicos de cada columna (útil para inspeccionar)
for col in df.columns:
    print(f"\nColumna: {col}")
    print(df[col].unique()[:10])  


Columna: Diabetes_012
[0. 2. 1.]

Columna: HighBP
[1. 0.]

Columna: HighChol
[1. 0.]

Columna: CholCheck
[1. 0.]

Columna: BMI
[40. 25. 28. 27. 24. 30. 34. 26. 33. 21.]

Columna: Smoker
[1. 0.]

Columna: Stroke
[0. 1.]

Columna: HeartDiseaseorAttack
[0. 1.]

Columna: PhysActivity
[0. 1.]

Columna: Fruits
[0. 1.]

Columna: Veggies
[1. 0.]

Columna: HvyAlcoholConsump
[0. 1.]

Columna: AnyHealthcare
[1. 0.]

Columna: NoDocbcCost
[0. 1.]

Columna: GenHlth
[5. 3. 2. 4. 1.]

Columna: MentHlth
[18.  0. 30.  3.  5. 15. 10.  6. 20.  2.]

Columna: PhysHlth
[15.  0. 30.  2. 14. 28.  7. 20.  3. 10.]

Columna: DiffWalk
[1. 0.]

Columna: Sex
[0. 1.]

Columna: Age
[ 9.  7. 11. 10.  8. 13.  4.  6.  2. 12.]

Columna: Education
[4. 6. 3. 5. 2. 1.]

Columna: Income
[3. 1. 8. 6. 4. 7. 2. 5.]


In [17]:
# Columnas diabetes = Target
df.rename(columns={'Diabetes_012': 'Diabetes'}, inplace=True)
df['Diabetes'] = df['Diabetes'].astype(int)  
df['Diabetes'] = df['Diabetes'].map({0: 'No', 1: 'Pre-Diabetes', 2: 'Diabetes'}).astype('category')

# High Blood Pressure
df['HighBP'] = df['HighBP'].astype(int)  
df['HighBP'] = df['HighBP'].map({0: 'No', 1: 'Yes'}).astype('category')

# High Cholesterol
df['HighChol'] = df['HighChol'].astype(int)  
df['HighChol'] = df['HighChol'].map({0: 'No', 1: 'Yes'}).astype('category')

# Cholesterol check in 5 years
df['CholCheck'] = df['CholCheck'].astype(int)  
df['CholCheck'] = df['CholCheck'].map({0: 'No', 1: 'Yes'}).astype('category')

# Smoker: have you smoked at least 100 cigarettes in your entire life?
df['Smoker'] = df['Smoker'].astype(int)  
df['Smoker'] = df['Smoker'].map({0: 'No', 1: 'Yes'}).astype('category')

# Stroke: do you now have or have you ever had a stroke? (accidente cerebrovascular)
df['Stroke'] = df['Stroke'].astype(int)  
df['Stroke'] = df['Stroke'].map({0: 'No', 1: 'Yes'}).astype('category')

# Heart disease or heart attack: 
df.rename(columns={'HeartDiseaseorAttack': 'HeartAttack'}, inplace=True)
df['HeartAttack'] = df['HeartAttack'].astype(int)  
df['HeartAttack'] = df['HeartAttack'].map({0: 'No', 1: 'Yes'}).astype('category')

# Pyshical activity in past 30 days
df['PhysActivity'] = df['PhysActivity'].astype(int)  
df['PhysActivity'] = df['PhysActivity'].map({0: 'No', 1: 'Yes'}).astype('category')

# Consume Fruit 1 or more times per day
df['Fruits'] = df['Fruits'].astype(int)  
df['Fruits'] = df['Fruits'].map({0: 'No', 1: 'Yes'}).astype('category')

# Consume veggies 1 or more times per day
df['Veggies'] = df['Veggies'].astype(int)  
df['Veggies'] = df['Veggies'].map({0: 'No', 1: 'Yes'}).astype('category')

# Alcohol Consumer: Heavy drinkers (adult men having more than 14 drinks per week and adult women having more than 7 drinks per week)
df.rename(columns={'HvyAlcoholConsump': 'Heavy_drinking'}, inplace=True)
df['Heavy_drinking'] = df['Heavy_drinking'].astype(int)  
df['Heavy_drinking'] = df['Heavy_drinking'].map({0: 'No', 1: 'Yes'}).astype('category')

# Health Care: Have any kind of health care coverage, including health insurance, prepaid plans such as HMO, etc.
df.rename(columns={'AnyHealthcare': 'HealthCare'}, inplace=True)
df['HealthCare'] = df['HealthCare'].astype(int)  
df['HealthCare'] = df['HealthCare'].map({0: 'No', 1: 'Yes'}).astype('category')

# Unaffordable_care: Was there a time in the past 12 months when you needed to see a doctor but could not because of cost?df.rename(columns={'AnyHealthcare': 'HealthCare'}, inplace=True)
df.rename(columns={'NoDocbcCost': 'Unaffordable_care'}, inplace=True)
df['Unaffordable_care'] = df['Unaffordable_care'].astype(int)  
df['Unaffordable_care'] = df['Unaffordable_care'].map({0: 'No', 1: 'Yes'}).astype('category')

#  General Health: would you say that in general your health is: 1-5: 1 = excellent 2 = very good 3 = good 4 = fair 5 = poor
df.rename(columns={'GenHlth': 'GeneralHealth'}, inplace=True)
df['GeneralHealth'] = df['GeneralHealth'].astype(int)  
df['GeneralHealth'] = df['GeneralHealth'].map({1: 'Excellent', 2: 'Very Good', 3: 'Good', 4: 'Fair', 5: 'Poor'}).astype('category')

#  Mental Health and physical health
df.rename(columns={'GenHlth': 'GeneralHealth'}, inplace=True)
df.rename(columns={'PhysHlth': 'PhysicalHealth'}, inplace=True)

# Difficulty walking : Do you have serious difficulty walking or climbing stairs?
df['DiffWalk'] = df['DiffWalk'].astype(int)  
df['DiffWalk'] = df['DiffWalk'].map({0: 'No', 1: 'Yes'}).astype('category')

# Sex: 0 = female, 1 = male
df['Sex'] = df['Sex'].astype(int)  
df['Sex'] = df['Sex'].map({0: 'Female', 1: 'Male'}).astype('category')

df.head(5)

,Diabetes,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartAttack,PhysActivity,Fruits,...,HealthCare,Unaffordable_care,GeneralHealth,MentHlth,PhysicalHealth,DiffWalk,Sex,Age,Education,Income
0,No,Yes,Yes,Yes,40.0,Yes,No,No,No,No,...,Yes,No,Poor,18.0,15.0,Yes,Female,9.0,4.0,3.0
1,No,No,No,No,25.0,Yes,No,No,Yes,No,...,No,Yes,Good,0.0,0.0,No,Female,7.0,6.0,1.0
2,No,Yes,Yes,Yes,28.0,No,No,No,No,Yes,...,Yes,Yes,Poor,30.0,30.0,Yes,Female,9.0,4.0,8.0
3,No,Yes,No,Yes,27.0,No,No,No,Yes,Yes,...,Yes,No,Very Good,0.0,0.0,No,Female,11.0,3.0,6.0
4,No,Yes,Yes,Yes,24.0,No,No,No,Yes,Yes,...,Yes,No,Very Good,3.0,0.0,No,Female,11.0,5.0,4.0


Vamos a entender la forma de nuestros dataset, los tipos de los atributos una vez corregidos, y si alguno de ellos contiene valores nulos:

In [20]:
print("📌 Dimensiones del dataset:")
print(df.shape)

summary = pd.DataFrame({
    'dtype': df.dtypes,
    'nulls': df.isnull().sum(),
})

print(summary)

📌 Dimensiones del dataset:
(253680, 22)
                      dtype  nulls
Diabetes           category      0
HighBP             category      0
HighChol           category      0
CholCheck          category      0
BMI                 float64      0
Smoker             category      0
Stroke             category      0
HeartAttack        category      0
PhysActivity       category      0
Fruits             category      0
Veggies            category      0
Heavy_drinking     category      0
HealthCare         category      0
Unaffordable_care  category      0
GeneralHealth      category      0
MentHlth            float64      0
PhysicalHealth      float64      0
DiffWalk           category      0
Sex                category      0
Age                 float64      0
Education           float64      0
Income              float64      0
